# Módulo 2: Extração em Massa de Anúncios das Páginas de Busca do Airbnb

## 1. O Objetivo: Construir um Dataset do Zero

Este notebook é o ponto de partida de todo o nosso projeto de dados. Sua missão é construir, do zero, um grande conjunto de dados (dataset) simulando um usuário que busca por acomodações no Airbnb.

### O Desafio
A análise de dados de aluguel por temporada depende de uma grande quantidade de informações. Como podemos coletar dados de preços e disponibilidade para centenas de datas diferentes, em múltiplos bairros, de forma automática? A resposta está na automação de buscas.

### A Solução: Um Robô de Busca e Coleta em Larga Escala
Este notebook funciona como um robô de busca. Nós o programamos com uma lista de locais e um período de tempo, e ele assume a tarefa de:
1.  Visitar o Airbnb e pesquisar por estadias em cada um dos locais.
2.  Para cada local, iterar dia a dia dentro dos meses definidos, realizando uma nova busca para cada data de check-in.
3.  Em cada página de resultados, extrair as informações principais de cada anúncio (ID, título, avaliação, preço, link).
4.  Navegar por todas as páginas de resultados ("Próximo", "Próximo", ...) até o fim.
5.  Salvar os dados coletados de forma incremental em um único arquivo CSV, criando a base para as próximas etapas de análise e enriquecimento de dados.

Vamos dar início à construção do nosso dataset.

## 2. Preparando as Ferramentas para a Grande Busca

Assim como no módulo anterior, o primeiro passo é importar nossa "caixa de ferramentas". As bibliotecas são as mesmas, pois as tarefas de automação web e manipulação de dados são semelhantes, mas aplicadas a um objetivo diferente: a coleta em massa a partir das páginas de busca.

In [ ]:
# --- Bibliotecas para Manipulação de Dados e Arquivos ---
import pandas as pd  # Essencial para organizar os dados coletados em uma tabela (DataFrame).
import os            # Usado para interagir com o sistema operacional, especificamente para verificar se o arquivo de saída já existe.
import re            # Biblioteca de Expressões Regulares, usada para extrair informações específicas de textos, como o ID do imóvel a partir do link.
import time          # Permite adicionar pausas (esperas) no código, crucial para esperar o carregamento das páginas.
from datetime import date, timedelta  # Usado para manipular e calcular datas, como o dia do check-in e do check-out.
import calendar      # Usado para obter o número de dias em um determinado mês e ano.

# --- Bibliotecas para Web Scraping e Automação ---
from selenium import webdriver  # A ferramenta principal que controla o navegador.
from selenium.webdriver.firefox.options import Options  # Permite configurar as opções do navegador (ex: modo headless).
from selenium.webdriver.common.by import By  # Usado para especificar como encontrar elementos na página (por ID, XPath, CSS Selector, etc.).
from selenium.webdriver.support.ui import WebDriverWait  # Permite criar esperas inteligentes, aguardando que uma condição específica aconteça.
from selenium.webdriver.support import expected_conditions as EC  # Contém a lista de condições possíveis para as esperas (ex: aguardar um elemento ficar visível).
from selenium.common.exceptions import TimeoutException, NoSuchElementException  # Classes de erro do Selenium que usamos para tratar exceções, como um elemento não ser encontrado.
from bs4 import BeautifulSoup  # Biblioteca para analisar (parse) o código HTML da página e facilitar a extração de dados dos anúncios.

## 3. Ensinando o Robô a Pesquisar e Coletar

Aqui está a função `buscar_e_extrair_airbnb`, o "cérebro" deste robô. Ela contém toda a lógica para realizar **uma busca completa** para um conjunto específico de parâmetros (local, datas e hóspedes).

Seu trabalho é:
1.  Construir a URL de busca correta.
2.  Acessar a página.
3.  Enquanto houver uma página de resultados, extrair os dados de todos os "cards" de anúncio.
4.  Clicar no botão "Próximo" para avançar.
5.  Repetir o processo até não haver mais páginas.
6.  Ao final, devolver todos os dados coletados nessa busca em uma tabela organizada (DataFrame).

In [ ]:
def buscar_e_extrair_airbnb(driver, local, data_checkin, data_checkout, numero_hospedes, max_paginas=None):
    """
    Função para buscar hospedagens no Airbnb usando uma sessão de navegador existente.
    Navega por páginas, extrai dados e retorna um DataFrame.
    """
    # Cria uma lista vazia que irá armazenar os dados de todos os anúncios encontrados nesta busca.
    dados_hospedagens = []

    try:
        # Formata as datas do formato DD/MM/AAAA para o formato AAAA-MM-DD, que é o padrão usado na URL do Airbnb.
        checkin_iso = f"{data_checkin[6:]}-{data_checkin[3:5]}-{data_checkin[:2]}"
        checkout_iso = f"{data_checkout[6:]}-{data_checkout[3:5]}-{data_checkout[:2]}"
        # Monta a URL de busca final com todos os parâmetros.
        url = (f"https://www.airbnb.com.br/s/{local}/homes?checkin={checkin_iso}"
               f"&checkout={checkout_iso}&adults={numero_hospedes}")

        print(f"Acessando a URL: {url}")
        # Comanda o navegador para acessar a URL construída.
        driver.get(url)

        # Inicia um contador para as páginas de resultados.
        pagina_atual = 1
        # Inicia um loop infinito que só será quebrado quando não houver mais páginas ou ocorrer um erro.
        while True:
            # Condição de parada opcional: se um número máximo de páginas foi definido e atingido, para a extração.
            if max_paginas is not None and pagina_atual > max_paginas:
                print(f"\nLimite de {max_paginas} página(s) atingido. Finalizando extração para esta data.")
                break

            print(f"\n--- Extraindo dados da página {pagina_atual} ---")

            # Bloco de espera inteligente: aguarda os 'cards' de anúncio aparecerem na tela.
            try:
                # Aguarda por até 20 segundos até que pelo menos um 'card' de anúncio esteja presente no HTML.
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='card-container']"))
                )
                # Adiciona uma pausa estática de 5 segundos para garantir que todos os dados dentro dos cards sejam carregados.
                time.sleep(5)
            except TimeoutException:
                # Se os 'cards' não carregarem em 20 segundos, exibe uma mensagem.
                print("Tempo de espera excedido. Não foi possível carregar os anúncios.")
                # Tenta verificar se a página exibiu uma mensagem de "Nenhum resultado".
                try:
                    no_results_element = driver.find_element(By.CSS_SELECTOR, "h1")
                    if "Nenhum resultado" in no_results_element.text:
                        print("A página indica 'Nenhum resultado' para os filtros aplicados.")
                except NoSuchElementException:
                    pass # Se não encontrar a mensagem, apenas continua.
                break # Interrompe o loop 'while' para esta busca, pois não há anúncios.

            # Pega o código-fonte HTML da página atual e o entrega ao BeautifulSoup.
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            # Encontra todos os elementos <div> que correspondem a um 'card' de anúncio.
            listings = soup.find_all('div', {'data-testid': 'card-container'})

            # Se a lista de anúncios estiver vazia, encerra o loop para esta busca.
            if not listings:
                print("Nenhum anúncio encontrado nesta página, finalizando.")
                break

            print(f"Encontrados {len(listings)} anúncios na página {pagina_atual}.")

            # Itera sobre cada 'card' de anúncio encontrado na página.
            for listing in listings:
                # --- Extração dos dados de cada card ---
                
                # Encontra a tag 'a' que contém o link do anúncio.
                link_tag = listing.find('a', href=True)
                # Constrói o link completo e verifica se a tag foi encontrada.
                link = "https://www.airbnb.com.br" + link_tag['href'] if link_tag and link_tag.get('href') else 'N/A'

                # Extrai o ID do imóvel a partir do link usando expressões regulares.
                imovel_id = 'N/A'
                if link != 'N/A':
                    id_match = re.search(r'/rooms/(\d+)', link) # Procura por '/rooms/' seguido de um ou mais dígitos.
                    if id_match:
                        imovel_id = id_match.group(1) # Pega apenas o grupo de dígitos.

                # Extrai o título do anúncio.
                title_div = listing.find('div', {'data-testid': 'listing-card-title'})
                title = title_div.text.strip() if title_div else 'N/A'

                # Extrai o tipo de acomodação a partir do título.
                tipo_acomodacao = 'N/A'
                if ' em ' in title:
                    tipo_acomodacao = title.split(' em ', 1)[0]

                # Extrai a nota e a quantidade de avaliações.
                nota_avaliacao, qtd_avaliacoes = 'N/A', 'N/A'
                # O nome da classe pode mudar, então buscamos por uma parte que costuma ser constante.
                rating_container = listing.find('span', class_=re.compile(r'r4a59j5'))
                if rating_container:
                    rating_span = rating_container.find('span', {'aria-hidden': 'true'})
                    if rating_span:
                        full_rating_text = rating_span.text.strip()
                        # Trata o caso especial de anúncios "Novos" que não têm nota.
                        if "Novo" in full_rating_text:
                            nota_avaliacao, qtd_avaliacoes = 'Novo', '0'
                        else:
                            # Usa regex para extrair a nota (pode ter vírgula ou ponto).
                            score_match = re.search(r'([\d,.]+)', full_rating_text)
                            if score_match: nota_avaliacao = score_match.group(1)
                            # Usa regex para extrair a quantidade de avaliações (número entre parênteses).
                            count_match = re.search(r'\((\d+)\)', full_rating_text)
                            if count_match: qtd_avaliacoes = count_match.group(1)

                # Extrai o preço total e a quantidade de noites.
                preco, qtd_noites = 'N/A', 'N/A'
                price_row = listing.find('div', {'data-testid': 'price-availability-row'})
                if price_row:
                    full_price_text = price_row.get_text(separator=' ').strip()
                    # Extrai o valor do preço.
                    preco_match = re.search(r'R\$\s*([\d.]+)', full_price_text)
                    if preco_match:
                        preco_str = preco_match.group(1).replace('.', '') # Remove o separador de milhar.
                        preco = f"R${preco_str}"
                    # Extrai o número de noites.
                    noites_match = re.search(r'(\d+)\s*noites', full_price_text)
                    if noites_match: qtd_noites = noites_match.group(1)

                # Adiciona um dicionário com todos os dados extraídos à nossa lista principal.
                dados_hospedagens.append({
                    'ID Imóvel': imovel_id, 'Título': title, 'Tipo de Acomodação': tipo_acomodacao,
                    'Data de Check-in': data_checkin, 'Data de Check-out': data_checkout,
                    'Número de Hóspedes': numero_hospedes, 'Preço total': preco, 'Total de Noites': qtd_noites,
                    'Avaliação': nota_avaliacao, 'Quantidade de Avaliações': qtd_avaliacoes, 'Link': link
                })

            # Tenta encontrar e clicar no botão "Próximo" para ir para a próxima página de resultados.
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Próximo']")
                driver.execute_script("arguments[0].click();", next_button) # Clica usando JavaScript.
                pagina_atual += 1 # Incrementa o contador da página.
            except NoSuchElementException:
                # Se o botão "Próximo" não for encontrado, significa que chegamos à última página.
                print("Não há mais páginas para extrair. Fim da extração para esta data.")
                break # Quebra o loop 'while'.
    except Exception as e:
        # Captura qualquer outro erro inesperado que possa acontecer durante a busca.
        print(f"Ocorreu um erro geral durante a extração: {e}")

    # Ao final, converte a lista de dicionários em um DataFrame do pandas e o retorna.
    return pd.DataFrame(dados_hospedagens)

## 4. Orquestrando a Coleta em Massa: A Execução Principal

Com nossa função de busca pronta, precisamos de um "maestro" para orquestrar a operação. Esta seção define os parâmetros da nossa grande busca e executa os loops que chamarão a função `buscar_e_extrair_airbnb` repetidamente para cada combinação de local e data.

### 4.1. Definindo os Parâmetros da Busca

Este é o nosso painel de controle. Aqui, definimos para quais locais, meses, ano, e com qual duração de estadia o nosso robô deve realizar as buscas. O nome do arquivo de saída também será gerado dinamicamente com base nestes parâmetros para fácil identificação.

In [ ]:
# --- PAINEL DE CONTROLE DA BUSCA ---

# Defina aqui a lista de locais que você deseja pesquisar.
locais_busca = [
    "Copacabana, Rio de Janeiro",
    "Ipanema, Rio de Janeiro",
    "Barra da Tijuca, Rio de Janeiro",
    "Leblon, Rio de Janeiro"
]
# Defina a lista de meses para a busca (números de 1 a 12).
meses_busca = [8, 9, 10, 11, 12]
# Defina o ano da busca.
ano_busca = 2025
# Defina o número de hóspedes.
hospedes = 1
# Defina a duração da estadia em noites.
duracao_estadia_em_noites = 4

# --- GERAÇÃO AUTOMÁTICA DO NOME DO ARQUIVO ---
# Cria um nome de arquivo descritivo e único, incluindo a data de hoje.
nome_arquivo = f"airbnb_dados_gerais_{hospedes}_hospede_{duracao_estadia_em_noites}_noites_{date.today().strftime('%Y_%m_%d')}.csv"

# --- INICIALIZAÇÃO DAS VARIÁVEIS DE CONTROLE ---
# Inicia a variável do driver como 'None'. O navegador só será aberto quando for realmente necessário.
driver = None
# Inicia um contador de iterações para controlar quando o navegador deve ser reiniciado.
iteration_counter = 0

# Imprime uma mensagem inicial para o usuário.
print("--- INICIANDO BUSCA EM MASSA ---")
print(f"Os resultados serão salvos progressivamente em: '{nome_arquivo}'")

### 4.2. A Grande Maratona: Iterando por Locais, Meses e Dias

Esta é a automação em sua forma mais pura. O código abaixo cria uma série de loops aninhados para garantir que todas as combinações de busca sejam executadas.

Imagine uma equipe de pesquisa com uma lista de tarefas. O código funciona como o gerente dessa equipe, instruindo-a a:
1.  Pegar o primeiro local (ex: "Copacabana").
2.  Para esse local, pegar o primeiro mês da lista (ex: Agosto).
3.  Para esse mês, pesquisar, um por um, todos os dias (de 1 a 31) como data de check-in.
4.  Repetir o processo para todos os meses e, em seguida, para todos os locais da lista.

Ele também gerencia a vida útil do navegador, reiniciando-o a cada 100 buscas para garantir a estabilidade do processo.

In [ ]:
# Loop 1: Itera sobre cada LOCAL da nossa lista de busca.
for local in locais_busca:
    # Loop 2: Itera sobre cada MÊS da nossa lista.
    for mes in meses_busca:
        # Obtém o número exato de dias no mês/ano atual (ex: 31 para Agosto, 30 para Setembro).
        num_dias_no_mes = calendar.monthrange(ano_busca, mes)[1]

        # Imprime um cabeçalho para indicar o início do processamento de um novo local/mês.
        print(f"\n{'=' * 60}")
        print(f"PROCESSANDO LOCAL: {local} | MÊS/ANO: {mes:02d}/{ano_busca}")
        print(f"{'=' * 60}")

        # Loop 3: Itera sobre cada DIA do mês atual.
        for dia in range(1, num_dias_no_mes + 1):
            # Incrementa o contador geral de iterações.
            iteration_counter += 1

            # Lógica para reiniciar o navegador a cada 100 buscas para liberar memória RAM.
            if iteration_counter > 1 and (iteration_counter - 1) % 100 == 0:
                if driver: # Verifica se existe um navegador aberto.
                    print(f"\n--- [Iteração {iteration_counter - 1}] Reiniciando o navegador para liberar recursos ---")
                    driver.quit() # Fecha o navegador.
                    driver = None # Define a variável como None para que um novo seja criado abaixo.

            # Se não houver um navegador ativo (seja no início ou após uma reinicialização), cria um novo.
            if driver is None:
                print("\n--- Iniciando uma nova sessão do navegador ---")
                options = Options()
                options.add_argument('--headless')
                options.add_argument('--disable-gpu')
                options.add_argument('--no-sandbox')
                options.add_argument('--disable-dev-shm-usage') # Configurações para otimizar a execução em ambientes de servidor/nuvem.
                options.set_preference("permissions.default.image", 2)
                options.set_preference("permissions.default.stylesheet", 2)
                options.set_preference("gfx.downloadable_fonts.enabled", False)
                options.set_preference("media.autoplay.enabled", False)
                driver = webdriver.Firefox(options=options)

            # Calcula as datas de check-in e check-out para a iteração atual.
            data_de_checkin = date(ano_busca, mes, dia)
            data_de_checkout = data_de_checkin + timedelta(days=duracao_estadia_em_noites)

            # Formata as datas para o padrão DD/MM/AAAA.
            checkin_str = data_de_checkin.strftime("%d/%m/%Y")
            checkout_str = data_de_checkout.strftime("%d/%m/%Y")

            # Lógica para verificar se o período da estadia inclui um fim de semana (sábado ou domingo).
            inclui_fim_de_semana = "Não"
            for i in range(duracao_estadia_em_noites + 1):
                dia_da_estadia = data_de_checkin + timedelta(days=i)
                if dia_da_estadia.weekday() >= 5: # 5=Sábado, 6=Domingo.
                    inclui_fim_de_semana = "Sim"
                    break # Se encontrou um, não precisa verificar os outros dias.

            print(f"\n--- Buscando dia {dia}/{num_dias_no_mes} para {local} | Check-in: {checkin_str} ---")

            # Chama a função principal de extração com os parâmetros da iteração atual.
            df_resultado_diario = buscar_e_extrair_airbnb(
                driver, local, checkin_str, checkout_str, hospedes
                # A linha abaixo pode ser descomentada para testes rápidos, limitando a 1 página por busca.
                # , max_paginas=1 
            )

            # Se a função retornou algum dado (o DataFrame não está vazio).
            if not df_resultado_diario.empty:
                # Adiciona as colunas 'Localização' e 'Inclui Fim de Semana' que são fixas para esta busca.
                df_resultado_diario['Localização'] = local
                df_resultado_diario['Inclui Fim de Semana'] = inclui_fim_de_semana

                # Define a ordem desejada para as colunas no arquivo CSV final.
                colunas_ordenadas = [
                    'Localização', 'ID Imóvel', 'Título', 'Tipo de Acomodação',
                    'Data de Check-in', 'Data de Check-out', 'Inclui Fim de Semana',
                    'Número de Hóspedes', 'Preço total', 'Total de Noites', 'Avaliação',
                    'Quantidade de Avaliações', 'Link'
                ]
                df_resultado_diario = df_resultado_diario[colunas_ordenadas]

                # Verifica se o arquivo de saída já existe para decidir se o cabeçalho deve ser escrito.
                escrever_cabecalho = not os.path.exists(nome_arquivo)

                # Anexa os resultados da busca atual ao arquivo CSV.
                df_resultado_diario.to_csv(
                    nome_arquivo,
                    mode='a', # 'a' significa 'append' (adicionar ao final do arquivo).
                    header=escrever_cabecalho, # Só escreve o cabeçalho se o arquivo não existir.
                    index=False, # Não salva o índice do DataFrame no arquivo.
                    encoding='utf-8-sig' # Codificação que melhora a compatibilidade com o Excel.
                )
                print(f"SUCESSO: {len(df_resultado_diario)} novos registros salvos em '{nome_arquivo}'")
            else:
                # Se a busca não retornou nenhum anúncio.
                print(f"AVISO: Nenhum resultado encontrado para {checkin_str} em {local}.")

# --- Finalização do Script ---
if driver: # Se, ao final de todos os loops, ainda houver um navegador aberto.
    print("\n--- Fechando a sessão final do navegador. ---")
    driver.quit() # Encerra o navegador e libera os recursos.

print("\n\n--- EXTRAÇÃO EM MASSA CONCLUÍDA ---")